In [3]:
import torch
import numpy as np 
import pandas as pd
import PIL

In [2]:
torch.cuda.is_available()

True

In [4]:
PIL.PILLOW_VERSION

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: PILLOW_VERSION is deprecated and will be removed in a future release. Use __version__ instead.
  """Entry point for launching an IPython kernel.


'7.1.2'

In [5]:
from google.colab import drive

In [6]:
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [7]:
! ls /content/gdrive/

MyDrive  Shareddrives


In [18]:
!unzip -q /content/gdrive/MyDrive/intel_image.zip -d seg_train
!unzip -q /content/gdrive/MyDrive/intel_image.zip -d seg_test

In [11]:
!ls seg_train

characters_illustration.png  sample_submission.csv  testset  train


In [13]:
import pickle 

from skimage import io
from tqdm import tqdm, tqdm_notebook
from PIL import Image
from torchvision import transforms, models
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from matplotlib import colors, pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)


In [15]:
data_mode = ['train', 'test', 'val']
image_size = 100
device = torch.device('cuda')

In [16]:
class ImageDataset(Dataset):
    def __init__(self, files, mode):
        super().__init__()
        # список файлов для загрузки
        self.files = sorted(files)
        # режим работы
        self.mode = mode

        if self.mode not in data_mode:
            print(f"{self.mode} is not correct; correct modes: {data_mode}")
            raise NameError

        self.len_ = len(self.files)
     
        self.label_encoder = LabelEncoder()

        if self.mode != 'test':
            self.labels = [path.parent.name for path in self.files]
            self.label_encoder.fit(self.labels)

            with open('label_encoder.pkl', 'wb') as le_dump_file:
                  pickle.dump(self.label_encoder, le_dump_file)
                      
    def __len__(self):
        return self.len_
      
    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image
  
    def __getitem__(self, index):
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
        ])
        x = self.load_sample(self.files[index])
        x = self._prepare_sample(x)
        x = np.array(x / 255, dtype='float32')
        x = transform(x)
        if self.mode == 'test':
            return x
        else:
            label = self.labels[index]
            label_id = self.label_encoder.transform([label])
            y = label_id.item()
            return x, y
        
    def _prepare_sample(self, image):
        image = image.resize((image_size, image_size))
        return np.array(image)

In [17]:
def imshow(inp, title=None, plt_ax=plt, default=False):
    """Imshow для тензоров"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt_ax.imshow(inp)
    if title is not None:
        plt_ax.set_title(title)
    plt_ax.grid(False)

In [ ]:
train_dir = Path('seg_train/')